In [13]:
from IPython.display import display, Math

# Define the boolean expression
clauses = [(1,3,2), (-1,2), (-2, 3)]
# clauses = [(1,2), (2,3), (1, 3)]
#clauses = [(1, 2, 3), (1, 2, -3), (1, -2, 3), (1, -2, -3), (-1, 2, 3), (-1, 2, -3), (-1, -2, 3)]

# Get a list of variables
variables = set()
for clause in clauses:
    for variable in clause:
        variables.add(abs(variable))

print("List of variables:")
display(Math(', '.join(f'x{i}' for i in sorted(variables))))

# Print the boolean expression
latex_clauses = []
for clause in clauses:
    clause_literals = [f'\\overline{{x{abs(lit)}}}' if lit < 0 else f'x{abs(lit)}' for lit in clause]
    latex_clause = '(' + ' \\lor '.join(clause_literals) + ')'
    latex_clauses.append(latex_clause)
    
print("Given boolean expression:", end='')
display(Math(' \\land '.join(latex_clauses)))

List of variables:


<IPython.core.display.Math object>

Given boolean expression:

<IPython.core.display.Math object>

In [12]:
# Converting a 3SAT expression to k-coloring:

# First (variables) amount of nodes will be the true-literals,
# second (variables) amoun will be the false-literals,
# each 3 after that will represent a clause
num_literals = len(variables)
literals = list(variables)
edges = []
for index, variable in enumerate(literals):
    edge = (index, index+num_literals)
    edges.append(edge)
literals_offset = num_literals*2
for index, clause in enumerate(clauses):
    clause_offset = index*3
    total_offset = literals_offset + clause_offset
    edge1 = (total_offset, total_offset+1)
    edge2 = (total_offset+1, total_offset+2)
    edge3 = (total_offset, total_offset+2)
    edges.append(edge1)
    edges.append(edge2)
    edges.append(edge3)
    for index, literal in enumerate(clause):
        inbound_edge = total_offset+index
        literalIndex = abs(literal)
        if (literal < 0):
            literal_node_offset = num_literals
        else:
            literal_node_offset = 0
        outbound_edge = literal_node_offset + literalIndex
        edges.append((inbound_edge, outbound_edge))
print(edges)

[(0, 3), (1, 4), (2, 5), (6, 7), (7, 8), (6, 8), (6, 1), (7, 3), (9, 10), (10, 11), (9, 11), (9, 4), (10, 2), (12, 13), (13, 14), (12, 14), (12, 5), (13, 3)]
